In [2]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
from pathos.multiprocessing import ProcessingPool
from pathos.multiprocessing import ProcessPool
import datetime
import os
import progressbar
import pickle

In [3]:
main_path = 'D:\Google Drive\내 드라이브\Project\LiveCommerse\FUCKUGOOGLE//'
driver_path = 'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe'

In [6]:
url = 'https://shoppinglive.naver.com/categories/dc:1?sort=AMOUNT'

chrome_options = Options()
# chrome_options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

driver.get(url)
time.sleep(3)

In [7]:
# chrome_driver_path = driver_path
# chrome_options = Options()
# chrome_options.add_argument("--headless")
# driver = webdriver.Chrome(chrome_driver_path, options=chrome_options)


In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
url_list = soup.select('div.VideoBoxWrapper_wrap_Usbk7.VerticalCardList_item_YPN88')
url_list

In [11]:
temp_df = pd.DataFrame()
Beauty_video_list = []
for j, i in enumerate(url_list):
    i = str(i)
    temp = i.split('href="')[1]
    video_list = temp.split('"><div')[0]
    item_name = temp.split('</path></g></svg><img alt="')[2]
    item_name = item_name.split('" class="CardThumbnail_image_d88Hz"')[0]
    channel_list = i.split('href="')[2]
    channel_name = channel_list.split('BroadcastUnderCard_name_+7+cK">')[1]
    channel_name = channel_name.split('</span><')[0]
    channel_list = channel_list.split('"><span class=')[0]
    channel_list = 'https://shoppinglive.naver.com' + channel_list
    if 'replays' in video_list:
        temp_df.loc[j, 'Video_url'] = video_list
        temp_df.loc[j, 'Item_Name'] = item_name
        temp_df.loc[j, 'Channel_url'] = channel_list
        temp_df.loc[j, 'Channel_Name'] = channel_name


In [20]:
temp_df.to_excel(main_path + 'Beauty/Beauty_data.xlsx', index=False)

In [4]:
df = pd.read_excel(main_path + 'Beauty/Beauty_data.xlsx')

In [5]:
df.head(2)

,Video_url,Item_Name,Channel_url,Channel_Name
0,https://view.shoppinglive.naver.com/replays/92...,닥터지 레드 블레미쉬 스페셜 진정 세트(크림 50ml+선 35ml),https://shoppinglive.naver.com/channels/10418,고운세상 닥터지
1,https://view.shoppinglive.naver.com/replays/90...,[공식몰]C E 페룰릭 세트/항산화 안티에이징 앰플(+나이트 항산화 에센스 3종),https://shoppinglive.naver.com/channels/70049,스킨수티컬즈


In [6]:
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [7]:
for i in range(len(df)):
    temp_df = df.loc[i]
    video_url = temp_df[0]
    channel_url = temp_df[2]
    
    driver.get(video_url)
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    video_view = soup.select('span.BroadcastStatusBadge_number_ZHAih')
    video_view = str(video_view)
    video_view = video_view.split('">')[1]
    video_view = video_view.split('</sp')[0]
    video_view = re.sub(r"[^0-9]", "", video_view)
    video_view = int(video_view)

    like_number = soup.select('span.LikeButton_number_IXgmK.__disable_double_tab')
    like_number = str(like_number)
    like_number = like_number.split('">')[1]
    like_number = like_number.split('</sp')[0]
    like_number = re.sub(r"[^0-9]", "", like_number)
    like_number = int(like_number)

    total_time = soup.select('time.VideoTime_size_l_mtdgH.VideoTime_position_right_ZNTiK')
    total_time = str(total_time)
    total_time = total_time.split('datetime="')[1]
    total_time = total_time.split('"')[0]
    
    try:
        price = soup.select('strong.DiscountPrice_price_eerER')
        price = str(price)
        price = price.split('">')[1]
        price = price.split('</st')[0]
        price = re.sub(r"[^0-9]", "", price)
        price = int(price)
    except:
        price = 0
    
    df.loc[i, 'Video_View'] = video_view
    df.loc[i, 'Like_Number'] = like_number
    df.loc[i, 'Video_Time'] = total_time
    df.loc[i, 'Item_Price'] = price

    driver.get(channel_url)
    time.sleep(6)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        Num_sub = str(soup.select('div.ProfileCreator_fan_lUrMY'))
        Num_sub = Num_sub.split('">')[1]
        Num_sub = Num_sub.split('명')[0]

        if '만' in Num_sub:
            Num_sub = re.sub(r"[^0-9]", "", Num_sub)
            Num_sub = float(Num_sub) * 10000
            Num_sub = int(Num_sub)
        else:
            Num_sub = re.sub(r"[^0-9]", "", Num_sub)
            Num_sub = int(Num_sub)
    except:
        Num_sub = 0
    
    try:
        Num_Video = str(soup.select('div.ItemCount_wrap_dYzmP'))

        Num_Video = (Num_Video.split('<')[1])
        Num_Video = (Num_Video.split('>')[1])
        Num_Video = (Num_Video.split(' ')[1])
    except:
        Num_Video = 0
        
    df.loc[i, 'Num_Subscriber'] = Num_sub
    df.loc[i, 'Num_Video'] = Num_Video
    
    df.to_excel(main_path + 'Beauty/Beauty_Total_Data.xlsx', index=False)